This is a notebook illustrating how numba and jit can speed up your code

In [1]:
import numpy as np
import timeit
from numba import jit

The following is some code for the Gillespie algorithim for a 2-d SIR model. I don't use jit here. My object here is to obtain the output of make_list and time it.

In [3]:
L =10
A = np.zeros((L,L))
A[int(L/2)-1,int(L/2)-1]=1

up_vector = np.roll(np.arange(L),-1)
down_vector = np.roll(np.arange(L),1)


def neighbours(A,B):
    N = np.zeros((L,L))
    ilist,jlist = np.where(A==0)
    for k in range(len(ilist)):
        i,j = ilist[k],jlist[k]
        N[i,j] = B[i,up_vector[j]]+B[i,down_vector[j]]+ \
        B[up_vector[i],j]+B[down_vector[i],j]+\
        B[up_vector[i],down_vector[j]]+ B[up_vector[i],up_vector[j]]+ \
        B[down_vector[i],down_vector[j]]+ B[down_vector[i],up_vector[j]]
    return N
    
    
    


def SIR_step(A,t):
    B = np.array(A)
    B[np.where(A!=1)]=0
    R = neighbours(A,B)
    w1 = np.sum(R)*r_I
    w2= np.sum(B)*rm
    W= w1+w2
    dt = -np.log(np.random.random_sample())/W
    t = t + dt
    y = W*np.random.random_sample()
    if y < w1:
        'gets infected'
        y= y/r_I
        index = np.where(np.cumsum(R)>y)[0][0]
        i,j =  np.unravel_index( index,(L,L))
        A[i,j] += 1
        
        
    else:
        'gets immune'
        y =(y -w1)/rm
        index = np.where(np.cumsum(B)>y)[0][0]
        i,j =  np.unravel_index( index,(L,L))
        A[i,j] += 1
    return A,t

rm = 1.1
T_max=400
t=0
r_I =0.5
N = L**2
Alist =[]
n=0

def t_exit(A):
    t=0 
    while np.where(A==1)[0].size >0:
        A,t = SIR_step(A,t)
    return t

n_trials =1000



def make_list(n_trials):
    t_exit_list=[]
    for n in range(n_trials):
        A = np.zeros((L,L))
        A[int(L/2)-1,int(L/2)-1]=1
        t_exit_list +=[t_exit(A)]
    return t_exit_list

def time_func():
    return make_list(100)

timeit.Timer(time_func).timeit(number=10)

20.195682365447283

Now, I precompile just the neighbours function by adding a single line to the top of it. 

In [15]:
L =10
A = np.zeros((L,L))
A[int(L/2)-1,int(L/2)-1]=1
up_vector = np.roll(np.arange(L),-1)
down_vector = np.roll(np.arange(L),1)

@jit(nogil=True)
def neighbours(A,B):
    N = np.zeros((L,L))
    ilist,jlist = np.where(A==0)
    for k in range(len(ilist)):
        i,j = ilist[k],jlist[k]
        N[i,j] = B[i,up_vector[j]]+B[i,down_vector[j]]+ \
        B[up_vector[i],j]+B[down_vector[i],j]+\
        B[up_vector[i],down_vector[j]]+ B[up_vector[i],up_vector[j]]+ \
        B[down_vector[i],down_vector[j]]+ B[down_vector[i],up_vector[j]]
    return N
    
    
    
#@jit(nogil=True)
def SIR_step(A,t):
    B = np.array(A)
    B[np.where(A!=1)]=0
    R = neighbours(A,B)
    w1 = np.sum(R)*r_I
    w2= np.sum(B)*rm
    W= w1+w2
    dt = -np.log(np.random.random_sample())/W
    t = t + dt
    y = W*np.random.random_sample()
    if y < w1:
        'gets infected'
        y= y/r_I
        index = np.where(np.cumsum(R)>y)[0][0]
        i,j =  np.unravel_index( index,(L,L))
        A[i,j] += 1
        
        
    else:
        'gets immune'
        y =(y -w1)/rm
        index = np.where(np.cumsum(B)>y)[0][0]
        i,j =  np.unravel_index( index,(L,L))
        A[i,j] += 1
    return A,t

rm = 1.1
T_max=400
t=0
r_I =0.5
N = L**2
Alist =[]
n=0

#@jit(nogil=True)
def t_exit(A):
    t=0 
    while np.where(A==1)[0].size >0:
        A,t = SIR_step(A,t)
    return t

n_trials =1000


#@jit(nogil=True)
def make_list(n_trials):
    t_exit_list=[]
    for n in range(n_trials):
        A = np.zeros((L,L))
        A[int(L/2)-1,int(L/2)-1]=1
        t_exit_list +=[t_exit(A)]
    return t_exit_list

#@jit
def time_func():
    return make_list(100)

timeit.Timer(time_func).timeit(number=10)

4.591692907735705

The speed has become nearly 5 times faster. Surpisingly, pre compiling the other functions, slightly increases the time for some reason. It's still atleast 3 times faster. 

So, what functions should you add that line to? The function that's called the most and maybe do some trial and error with it.